In [1]:
import pandas as pd

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

# mdict = {"CAT " : CatBoostClassifier(verbose = False), "LGBM " : LGBMClassifier(), "RF " : RandomForestClassifier(),
#         "LR " : LogisticRegression(solver = "liblinear"), "SGD " : SGDClassifier()}
mdict = {"RF " : RandomForestClassifier(),
        "LR " : LogisticRegression(solver = "liblinear"), "SGD " : SGDClassifier()}

tree_grid1 = {"n_estimators" : [20, 40, 60, 80, 100],
            "learning_rate" : [0.01, 0.05, 0.1]}
tree_grid2 = {"n_estimators" : [20, 40, 60, 80, 100],
             "max_features" : ["log2", "sqrt"]}
linear_grid1 = {"penalty" : ["l2", "l1"], 
                "C" : [0.6, 0.7, 0.8, 0.9, 1.0]}
linear_grid2 = {"loss" : ["modified_huber", "squared_hinge", "perceptron"],
               "penalty" : ["l2", "l1"]}

pdict = {0 : tree_grid1, 1 : tree_grid1, 2 : tree_grid2, 3 : linear_grid1, 4 : linear_grid2}

for m in mdict.keys():
    idx = 2
    for l1 in ["o", "v"]:
        for l2 in ["i", "b", "f"]:
            for n1 in range(1, 5):
                for n2 in range(0, 10):
                    if l2 == "b": n2 = 0
                    DATA_PATH = m + l1 + l2 + "_train" + str(n1) + "-" + str(n2) + ".csv"
                    DATA_PATH2 = l1 + l2 + "_test" + str(n1) + ".csv"
                    
                    train = pd.read_csv(DATA_PATH)
                    test = pd.read_csv(DATA_PATH2)
                    
                    if "Unnamed: 0" in train.columns: train.drop(["Unnamed: 0"], axis = 1, inplace = True)
                    if "encode_YR" in train.columns: train.drop(["encode_YR"], axis = 1, inplace = True)
                
                    test = test.loc[:, train.columns]
                    
                    model = mdict[m]
                    X, y = train.drop(["target"], axis = 1), train["target"]
                    X_test, y_test = test.drop(["target"], axis = 1), test["target"]
                    grid = GridSearchCV(model, param_grid = pdict[idx], cv = 3, refit = True)
                    grid.fit(X, y)
                    _pred = grid.best_estimator_.predict(X_test)
                    mat = confusion_matrix(y_test, _pred)
                    mat = pd.DataFrame(mat)
                    mat.to_csv(DATA_PATH[:-4] + "-res.csv")
    idx += 1

/opt/anaconda3/envs/tf/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/anaconda3/envs/tf/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/anaconda3/envs/tf/lib/python3.7/site-packages/sklearn/ensemble/_forest.py", line 304, in fit
    accept_sparse="csc", dtype=DTYPE)
  File "/opt/anaconda3/envs/tf/lib/python3.7/site-packages/sklearn/base.py", line 432, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "/opt/anaconda3/envs/tf/lib/python3.7/site-packages/sklearn/utils/validation.py", line 72, in inner_f
    return f(**kwargs)
  File "/opt/anaconda3/envs/tf/lib/python3.7/site-packages/sklearn/utils/validation.py", line 802, in check_X_y
    estimator=estimator)
 

ValueError: at least one array or dtype is required